#VIDEOMAE

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys 

sys.path.append("G:\\CODE\\VIDEOMAE\\videomae")

In [7]:
!python -m pip install -r ../requirements.txt
!python -m pip install -e ../script/mltools

Obtaining file:///G:/CODE/VIDEOMAE/videomae/script/mltools
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for mltools (pyproject.toml): started
  Building editable for mltools (pyproject.toml): finished with status 'done'
  Created wheel for mltools: filename=mltools-0.0.1-0.editable-py3-none-any.whl size=1436 sha256=cbc4cc7093d42a0618612bd13bf2318c85da6d7337cd3764046cd56b94a49b3a
  Stored in directory: C:\Users\ninhn\App

In [9]:
import os
from transformers import Trainer
from mltools.utl.file_util import get_subfolders
from videomae.eval.inference import run_inference
from videomae.train.dataset import load_test_set, load_train_set, load_val_set
from videomae.train.compute import collate_fn, compute_metrics
from videomae.train.model import get_image_processor, get_model
from videomae.train.trainer import get_train_args
from videomae.train.transform import init_train_transform, init_val_transform
from videomae.train.utl import display_gif, print_sample_info
from videomae.config import clip_duration, dataset_dir, model_ckpt, model_local, new_model_name

In [10]:
subset = get_subfolders(dataset_dir)
cls_name = get_subfolders(os.path.join(dataset_dir, subset[0]))
id2label = {i: cls for i, cls in enumerate(cls_name)}
label2id = {cls: i for i, cls in id2label.items()}

In [11]:
image_processor = get_image_processor(model_ckpt)
model = get_model(model_local, label2id, id2label)

In [12]:
train_transform = init_train_transform()
val_transform = init_val_transform()

In [14]:
train_dataset = load_train_set(dataset_dir, clip_duration, train_transform)
val_dataset = load_val_set(dataset_dir, clip_duration, val_transform)
test_dataset = load_test_set(dataset_dir, clip_duration, val_transform)

print("Number of video: ", train_dataset.num_videos, val_dataset.num_videos, test_dataset.num_videos)

In [15]:
sample_video = next(iter(train_dataset))
print("Key of a sample data", sample_video.keys())
print_sample_info(sample_video, id2label)

Key of a sample data dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])
video torch.Size([3, 16, 224, 224])
video_name Rec_11-21-21_018.avi
video_index 158
clip_index 0
aug_index 0
label 0
Video label: be-khoa-xe


In [16]:
train_args = get_train_args(train_dataset)
trainer = Trainer(
    model,
    train_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

g:\CODE\VIDEOMAE\videomae\.env\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
trainer.train()
trainer.evaluate(test_dataset)

  0%|          | 0/848 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
trainer.save_model(new_model_name)
test_results = trainer.evaluate(test_dataset)
trainer.log_metrics("test", test_results)
trainer.save_metrics("test", test_results)
trainer.save_state()

In [ ]:
trainer.push_to_hub()
print(f"trained model {new_model_name}")

In [ ]:
trained_model = get_model(new_model_name, label2id, id2label)
sample_test_video = next(iter(test_dataset))
print_sample_info(sample_test_video)

In [ ]:
logits = run_inference(trained_model, sample_test_video)
display_gif(sample_test_video["video"])

In [ ]:
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

#THE END 